In [ ]:
# default_exp preprocessing

In [ ]:
#export

import srsly
from ssda_nlp.xml_parser import *
import pandas as pd

In [ ]:
#export

def xml_to_jsonl(path_to_xml_transcription):
    xml_transcription = open(path_to_xml_transcription, 'r', encoding="utf-8")
    prodigy_input = open(path_to_xml_transcription[:path_to_xml_transcription.find(".xml")] + ".jsonl", 'w', encoding="utf-8")
    
    in_entry = False
    current_entry = ''
    
    for line in xml_transcription:
        if "<entry>" in line:
            in_entry = True            
        elif in_entry and ("</entry>" in line):
            current_entry += line[:line.find("</entry>")]
            in_entry = False            
            prodigy_input.write("{\"text\":\"" + current_entry + "\"}\n")
            current_entry = ''
        elif in_entry:
            if ((line[len(line) - 1] == '\n') or (line[len(line) - 1] == '\r')) and (line[len(line) - 2] == '-'):
                current_entry += line[:len(line) - 2]
            elif line[len(line) - 1] == '-':
                current_entry += line[:len(line) - 1]
            elif (line[len(line) - 1] == '\n') or (line[len(line) - 1] == '\r'):
                current_entry += line[:len(line) - 1] + ' '
            else:
                current_entry += line
            
    xml_transcription.close()
    prodigy_input.close()
    
    return

In [ ]:
#no_test

xml_to_jsonl("transcriptions/matanzas_baut_1.xml")

In [ ]:
#export

def parse_annotation(path_to_annotation):   
    annotation = srsly.read_jsonl(path_to_annotation)
    
    spans = []
    texts = []
    
    for entry in annotation:        
        texts.append(entry["text"])
        temp = []
        if "spans" in entry:            
            for span in entry["spans"]:
                temp.append([span["start"], span["end"], span["label"]])
        spans.append(temp)
    
    #build list of unique entries and list of empty annotation dictionaries for each    
    annot_ls = []
    
    for text in texts:        
        annot_ls.append({"entities":[]})
            
    #populate annotation dictionaries
    for i in range(len(texts)):
        for span in spans[i]:        
            annot_ls[i]["entities"].append((int(span[0]), int(span[1]), span[2]))       
        
    #build list of tuples
    tuples = []    
    for i in range(len(texts)):
        tuples.append((texts[i], annot_ls[i]))
        
    return tuples       

In [ ]:
#no_test

print(parse_annotation("transcriptions/matanzas_baut_1_annot.jsonl")[0])

('[margin]: Juana. Esc.va Domingo veinte y dos de [roto] y nueve yo Thomas de Orvera baptize, y pusse [roto] s.tos oleos a Juana de nacion Mina esclava de[roto] Juan Joseph de Justis fueron sus P.P. Joseph Salcedo y Ana de Santiago su mugger, y lo firmé. [signed]: Thomas de Orvera ', {'entities': [(10, 16, 'PER'), (17, 23, 'CHAR'), (24, 62, 'DATE'), (66, 82, 'PER'), (121, 126, 'PER'), (137, 141, 'CHAR'), (142, 149, 'CHAR'), (159, 180, 'PER'), (197, 211, 'PER'), (214, 229, 'PER'), (263, 279, 'PER')]})


In [ ]:
#export

def prodigy_output_to_collated_df(path_to_annotation):
    tuples = parse_annotation(path_to_annotation)
    entry_nos = []
    entry_texts = []
    entities = []
    starts = []
    ends = []
    labels = []
    entry = 1
    for tup in tuples:
        for entity in tup[1]["entities"]:
            entry_nos.append(entry)
            entry_texts.append(tup[0])
            entities.append(tup[0][entity[0]:entity[1]])
            starts.append(entity[0])
            ends.append(entity[1])
            labels.append(entity[2])
        entry += 1
    
    collated_dict = {"entry_no": entry_nos, "text": entry_texts, "entity": entities, "start": starts, "end": ends, "label": labels}
    
    collated_df = pd.DataFrame(collated_dict)    
    
    return collated_df

In [ ]:
#no_test

mat_xml_df = prodigy_output_to_collated_df("transcriptions/matanzas_baut_1_annot.jsonl")
mat_xml_df.head()

,entry_no,text,entity,start,end,label
0,1,[margin]: Juana. Esc.va Domingo veinte y dos d...,Juana.,10,16,PER
1,1,[margin]: Juana. Esc.va Domingo veinte y dos d...,Esc.va,17,23,CHAR
2,1,[margin]: Juana. Esc.va Domingo veinte y dos d...,Domingo veinte y dos de [roto] y nueve,24,62,DATE
3,1,[margin]: Juana. Esc.va Domingo veinte y dos d...,Thomas de Orvera,66,82,PER
4,1,[margin]: Juana. Esc.va Domingo veinte y dos d...,Juana,121,126,PER


In [ ]:
#no_test

from nbdev.export import notebook2script
notebook2script()

Converted 31-collate-xml-entities-spans.ipynb.
Converted 33-split-data.ipynb.
Converted 41-generic-framework-for-spacy-training.ipynb.
Converted 42-all.ipynb.
Converted 42-initial-model.ipynb.
Converted 51-data-preprocessing.ipynb.
Converted 52-unstructured-to-markup.ipynb.
Converted 53-markup-to-spacy.ipynb.
Converted 61-prodigy-output-training-demo.ipynb.
